
# Explicación de Q-Learning

### Introducción: De la Búsqueda Aleatoria con Memoria a Q-Learning

Comenzaste con una búsqueda aleatoria con memoria, donde el agente recordaba caminos que funcionaban. Esto funcionaba bien en situaciones simples, pero tenía dificultades cuando el juego se volvía complicado, como cuando el Lago Congelado se volvía resbaladizo. Si el agente resbalaba, el camino recordado se volvía inútil y el agente podría confundirse o fallar.

### ¿Qué Hace Diferente a Q-Learning?

Q-Learning es más inteligente porque no solo memoriza caminos; aprende a tomar buenas decisiones en cualquier situación:

- **Generalización**: Q-Learning ayuda al agente a descubrir los mejores movimientos, incluso si las cosas no salen como se planeó, como resbalarse en el hielo.
  
- **Adaptabilidad**: El agente puede ajustarse a los cambios y seguir tomando decisiones inteligentes, incluso si termina en un lugar diferente al esperado.

- **Aprendizaje a Partir de la Experiencia**: El agente aprende de cada movimiento, gane o pierda, mejorando cada vez que juega.

### Fundamentos de Q-Learning

Q-Learning ayuda al agente a mejorar en un juego utilizando una tabla Q, que es como una hoja de trucos. La tabla comienza vacía, pero a medida que el agente juega, se llena con puntuaciones que muestran qué movimientos son buenos y cuáles no lo son.

#### La Ecuación de Aprendizaje

El agente actualiza la tabla Q usando esta ecuación:

```python
q_table[state, action] = q_table[state, action] + learning_rate * (
    reward + discount_rate * np.max(q_table[new_state, :]) - q_table[state, action]
)
```

Esta ecuación ayuda al agente a decidir cómo mejorar su hoja de trucos en función de lo que sucedió después de realizar un movimiento.

#### Exploración vs. Explotación

Al principio, el agente intenta movimientos aleatorios para aprender qué funciona (exploración). A medida que se vuelve más inteligente, usa lo que sabe para tomar mejores decisiones (explotación). Con el tiempo, explora menos y se basa más en lo que ha aprendido. Así es como se ve esto en el código:

```python
epsilon = 1  # Comenzar con exploración completa
epsilon_decay_rate = 0.0001  # Disminuir lentamente la exploración
rand_num_gen = np.random.default_rng()

# Durante el juego
if rand_num_gen.random() < epsilon:
    action = env.action_space.sample()  # Explorar: elegir una acción aleatoria
else:
    action = np.argmax(q_table[state, :])  # Explotar: elegir la mejor acción conocida
```

A medida que el agente aprende, el valor de `epsilon` disminuye:

```python
epsilon = max(epsilon - epsilon_decay_rate, 0)
```

Esto significa que el agente explora menos y comienza a tomar decisiones basadas en lo que ha aprendido.

### Funciones de Numpy en Q-Learning

- **np.zeros()**: Crea la tabla Q, comenzando con ceros.

```python
q_table = np.zeros([env.observation_space.n, env.action_space.n])  # Inicializar tabla Q
```

- **rand_num_gen.random()**: Ayuda al agente a decidir si explora o explota.

```python
if rand_num_gen.random() < epsilon:
    action = env.action_space.sample()  # Acción aleatoria para exploración
```

- **np.argmax()**: Encuentra el mejor movimiento basado en lo que ha aprendido el agente.

```python
action = np.argmax(q_table[state, :])  # Mejor acción conocida para explotación
```

- **np.max()**: Mira hacia adelante para encontrar la mejor recompensa posible desde la nueva posición del agente.

```python
reward + discount_rate * np.max(q_table[new_state, :])
```

# Asignación: implementar la ecuación de actualización de la tabla Q y ajustar los hiperparámetros

A continuación se muestra el código incompleto para un algoritmo de Q-learning. Para esta asignación, primero debes implementar la ecuación de Q-learning.

En segundo lugar, ajusta los hiperparámetros y observa los resultados ejecutando las visualizaciones en las últimas celdas.


In [ ]:
import gymnasium as gym
import numpy as np

# Setup Frozen Lake environment
env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=False, render_mode='rgb_array')

# Initialize variables for visualization
images = []
q_table_states = []

# Initialize Q-table with zeros
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters with reasonable tuning ranges
learning_rate = 0.1  # Learning rate (0.1 to 0.9)
discount_rate = 0.8  # Discount rate (0.8 to 0.99)
epsilon = 1  # Exploration rate (0.1 to 1)
epsilon_decay_rate = 0.009  # Exploration decay rate (0.0001 to 0.01)
rand_num_gen = np.random.default_rng()
episodes = 1000  # Number of episodes (500 to 10,000)

# Track rewards per episode
rewards_per_episode = np.zeros(episodes)

for episode in range(episodes):
    state = env.reset()[0]
    terminated, truncated = False, False

    while not terminated and not truncated:
        # Exploration vs. Exploitation
        if rand_num_gen.random() < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(q_table[state, :])  # Exploit

        new_state, reward, terminated, truncated, _ = env.step(action)

        # Step 1: Implement Q-learning equation

        # Visualization save only the first and last 100 episodes
        if episode <= 100 or episode > episodes - 100:
            images.append(env.render())
            q_table_states.append(q_table.copy())

        state = new_state

    # Decay exploration rate
    epsilon = max(epsilon - epsilon_decay_rate, 0)

    # Slow down learning when exploration ends
    if epsilon == 0:
        learning_rate = 0.0001

    # Track rewards for successful episodes
    if reward == 1:
        rewards_per_episode[episode] = 1

# Calculate the sum of rewards over the last 100 episodes
sum_rewards = np.zeros(episodes)
for t in range(episodes):
    sum_rewards[t] = np.sum(rewards_per_episode[max(0, t-100):(t+1)])

env.close()

### Results

In [ ]:
from matplotlib import pyplot as plt

plt.plot(sum_rewards)
plt.savefig('frozen_lake8x8.png')

In [ ]:
import sys
import os

# Add the parent directory to sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.plotting import plot_q_values_grid

plot_q_values_grid(q_table, 4)

In [ ]:
import sys
import os
from IPython.display import HTML

# Agrega el directorio principal a sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.plotting import display_video

video = display_video(images[-20:], interval=50)
HTML(video.to_jshtml())